In [1]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredExcelLoader

load_dotenv()
import os
api_key = os.getenv("OPENAI_API_KEY")

xslx_paths = 'test용사전엑셀.xlsx'
xslx_loader = UnstructuredExcelLoader(xslx_paths, mode="elements")

# 문서 로드
docs = xslx_loader.load()

# 문서 출력
print(docs[0].page_content[:200])

한국어 영어 개 dog 고양이 cat 바나나 banana 커피 coffee 안경 glasses 바위 rock 시험 test 저장 save 긍정적인 positve 산독끼 Zinrabit 꼬라니 Zingorani


In [ ]:
from langchain.vectorstores import MongoDBAtlasVectorSearch
from pymongo import MongoClient

# MongoDB Atlas 연결 설정
# MONGODB_ATLAS_URI = ""
# client = MongoClient(MONGODB_ATLAS_URI)

In [3]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in docs])

vectorstore = FAISS.from_documents(
    documents=semantic_chunks, 
    embedding=embeddings
)

# mongoDB 활용시
# vectorstore = MongoDBAtlasVectorSearch(
#     collection_name="semantic_chunks",
#     embedding=embeddings,
#     client=client
# )

vectorstore.add_documents(semantic_chunks)
retriever = vectorstore.as_retriever()




In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# Embedding 설정 및 chunking
embeddings = OpenAIEmbeddings()
semantic_chunker = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
semantic_chunks = semantic_chunker.create_documents([d.page_content for d in docs])

# FAISS vectorstore 생성
vectorstore = FAISS.from_documents(documents=semantic_chunks, embedding=embeddings)

# 번역을 위한 PromptTemplate 및 Chain 설정
trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

translation_prompt = PromptTemplate(
    input_variables=["text"],
    template="다음 텍스트를 영어로 번역해주세요: {text}"
)

translation_chain = LLMChain(
    llm=trans_llm,
    prompt=translation_prompt
)

C:\Users\201\AppData\Local\Temp\ipykernel_12816\144438238.py:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\201\AppData\Local\Temp\ipykernel_12816\144438238.py:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
C:\Users\201\AppData\Local\Temp\ipykernel_12816\144438238.py:24: LangCh

In [ ]:

# 쿼리
query = "산독끼는 꼬라니를 좋아해. 바나나도 좋아해"

# Step 1: 쿼리에서 단어를 추출하고 vectorstore에서 관련 문서 검색
query_words = query.split(" ")  # 쿼리에서 단어들을 추출
relevant_texts = []

# query 단어에 대해 vectorstore에서 관련 문서 찾기
for word in query_words:
    search_results = vectorstore.similarity_search(word, k=5)  # 해당 단어와 관련된 문서 검색
    for result in search_results:
        # 검색된 결과에서 실제 텍스트를 relevant_texts에 추가
        relevant_texts.append(result.page_content)

# Step 2: 검색된 문서들을 하나로 결합 (검색된 관련 문서만 결합)
# query에 포함된 단어들만 포함된 부분을 추출하여 연결
filtered_relevant_texts = []
for text in relevant_texts:
    if any(word in text for word in query_words):
        filtered_relevant_texts.append(text)

# 관련된 문서들을 하나로 결합
combined_relevant_text = " ".join(filtered_relevant_texts)  # 관련 문서들을 하나로 결합

# Step 3: 관련 문서들을 번역
translation_result = translation_chain.run(text=combined_relevant_text)

# 결과 출력
print("원본 쿼리:", query)
print("검색된 관련 텍스트:", combined_relevant_text)
print("번역 결과:", translation_result)

C:\Users\201\AppData\Local\Temp\ipykernel_12816\2160264360.py:17: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  translation_result = translation_chain.run(text=combined_relevant_text)


원본 쿼리: 산독끼는 꼬라니를 좋아해. 바나나도 좋아해
검색된 관련 텍스트: 한국어 영어 개 dog 고양이 cat 바나나 banana 커피 coffee 안경 glasses 바위 rock 시험 test 저장 save 긍정적인 positve 산독끼 Zinrabit 꼬라니 Zingorani 한국어 영어 개 dog 고양이 cat 바나나 banana 커피 coffee 안경 glasses 바위 rock 시험 test 저장 save 긍정적인 positve 산독끼 Zinrabit 꼬라니 Zingorani 한국어 영어 개 dog 고양이 cat 바나나 banana 커피 coffee 안경 glasses 바위 rock 시험 test 저장 save 긍정적인 positve 산독끼 Zinrabit 꼬라니 Zingorani 한국어 영어 개 dog 고양이 cat 바나나 banana 커피 coffee 안경 glasses 바위 rock 시험 test 저장 save 긍정적인 positve 산독끼 Zinrabit 꼬라니 Zingorani 한국어 영어 개 dog 고양이 cat 바나나 banana 커피 coffee 안경 glasses 바위 rock 시험 test 저장 save 긍정적인 positve 산독끼 Zinrabit 꼬라니 Zingorani
번역 결과: Here is the translated text in English:

Korean English
Dog dog
Cat cat
Banana banana
Coffee coffee
Glasses glasses
Rock rock
Test test
Save save
Positive positive
산독끼 Zinrabit
꼬라니 Zingorani

Korean English
Dog dog
Cat cat
Banana banana
Coffee coffee
Glasses glasses
Rock rock
Test test
Save save
Positive positive
산독끼 Zinrabit
꼬라니 Zingorani

Korean English
Dog dog

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


# OpenAI 모델 설정
trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 번역을 위한 PromptTemplate 설정
translation_prompt = PromptTemplate(
    input_variables=["words"],
    template="다음 단어들을 영어로 번역해주세요: {words}"
)

# LLMChain 생성
translation_chain = LLMChain(
    llm=trans_llm,
    prompt=translation_prompt
)

# 번역 요청할 단어들
words_to_translate = "산독끼, 꼬라니, 바나나, 개, 고양이, 커피, 안경, 바위, 시험"

# 번역 실행
translation_result = translation_chain.run(words=words_to_translate)

# 결과 출력
print("원본 단어들:", words_to_translate)
print("번역 결과:", translation_result)

C:\Users\201\AppData\Local\Temp\ipykernel_2728\4020573920.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  translation_result = translation_chain.run(words=words_to_translate)


원본 단어들: 산독끼, 꼬라니, 바나나, 개, 고양이, 커피, 안경, 바위, 시험
번역 결과: 다음 단어들의 영어 번역은 다음과 같습니다:

- 산독끼: mountain goat (산악 염소)
- 꼬라니: (이 단어는 특정한 의미가 없거나 잘 알려지지 않은 단어입니다. 추가적인 맥락이 필요합니다.)
- 바나나: banana
- 개: dog
- 고양이: cat
- 커피: coffee
- 안경: glasses
- 바위: rock
- 시험: exam (또는 test)

꼬라니에 대한 추가 정보가 필요하면 말씀해 주세요!
